In [1]:
import json
from tqdm import tqdm
from pathlib import Path
from collections import OrderedDict, defaultdict
import time

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [6]:
from pathlib import Path
import os
root_path =  Path(Path.cwd()).resolve()
owod_tasks_path = root_path / 'datasets_utils' / 'owod' / 'tasks'

In [9]:
with open(owod_tasks_path / 't1_train.txt', 'r') as f:
    train_lines = f.readlines()

In [12]:
owod_tasks_path / 't1_train.txt'

PosixPath('/groups/tri110414/yolo-pruebas/datasets_utils/owod/tasks/t1_train.txt')

In [59]:
split = 'train'  # 'train' or 'val'
video_number = 1  # 0-199 in train, 0 to 6 in val
frame_position_number = 0  # from the first (0) to last frame from each video...

In [64]:
# Now we want to count the number of boxes per class
root_path =  Path("/home/tri110414/nfs_home/datasets/OAK/")
images_folder_path = root_path / "images" / split
labels_folder_path = root_path / "labels" / split
videos_folder_path = root_path / split / "Videos" 

classes_json_path = 'datasets_utils/oak/oak_classes.json'
classes_dict = json.load(open(classes_json_path))

video_names_list = [d.name for d in sorted(images_folder_path.iterdir()) if d.is_dir()]
video_name = video_names_list[video_number]

frames_path_list = sorted((images_folder_path / video_name).iterdir())

## Check the number of annotated frames w.r.t. the total number of frames

In [83]:
# Check the number of annotated frames w.r.t. the total number of frames
for video_name in video_names_list:
    frames_path_list = sorted((images_folder_path / video_name).iterdir())

    # Get the video path
    video_path = videos_folder_path / f"{video_name}.mp4"
    cap = cv2.VideoCapture(video_path.as_posix())
    # Read the total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Release the video capture object
    cap.release()

    # print(f"Video: {video_name}")
    # print(f"Number of frames: {len(frames_path_list)}")
    # print(f'Total number of frames: {total_frames}')
    # print(f'Percentage of annotated frames: {len(frames_path_list)/total_frames*100:.2f}%')
    # print()
    
    print(f'Video: {video_name}. % of annotated frames: {len(frames_path_list)/total_frames*100:.2f}% ({len(frames_path_list)}/{total_frames})')

Video: 20140913_172950_458. % of annotated frames: 1.57% (397/25361)
Video: 20140930_114708_495. % of annotated frames: 1.57% (290/18488)
Video: 20141001_163756_772. % of annotated frames: 1.56% (260/16630)
Video: 20141004_171007_314. % of annotated frames: 1.57% (594/37869)
Video: 20141006_132312_894. % of annotated frames: 1.55% (450/29036)
Video: 20141008_154205_152. % of annotated frames: 1.56% (489/31248)
Video: 20141009_185447_957. % of annotated frames: 1.57% (299/19031)
Video: 20141011_120104_835. % of annotated frames: 1.54% (420/27237)
Video: 20141011_155639_773. % of annotated frames: 1.57% (267/17056)
Video: 20141013_180629_736. % of annotated frames: 1.57% (392/25046)
Video: 20141015_161746_653. % of annotated frames: 1.57% (552/35189)
Video: 20141016_180651_373. % of annotated frames: 1.56% (423/27032)
Video: 20141019_150508_959. % of annotated frames: 1.57% (764/48704)
Video: 20141019_155555_270. % of annotated frames: 1.56% (282/18045)
Video: 20141019_164919_101. % of a

## Check ocurrence vs prevalence

In [84]:
classes_dict

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'dining table': 10,
 'dog': 11,
 'horse': 12,
 'motorcycle': 13,
 'person': 14,
 'potted plant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'monitor': 19,
 'stroller': 20,
 'cabinet': 21,
 'door': 22,
 'curtain': 23,
 'painting': 24,
 'shelf': 25,
 'transformer': 26,
 'fence': 27,
 'desk': 28,
 'bridge': 29,
 'lamp': 30,
 'dome': 31,
 'railing': 32,
 'cushion': 33,
 'box': 34,
 'column': 35,
 'signboard': 36,
 'tactile paving': 37,
 'counter': 38,
 'sink': 39,
 'barrier': 40,
 'refrigerator': 41,
 'stairs': 42,
 'case': 43,
 'crutch': 44,
 'graffiti': 45,
 'coffee table': 46,
 'toilet': 47,
 'book': 48,
 'bench': 49,
 'road barrier gate': 50,
 'palm': 51,
 'fruit': 52,
 'computer': 53,
 'arcade machine': 54,
 'parking meter': 55,
 'light': 56,
 'truck': 57,
 'awning': 58,
 'streetlight': 59,
 'booth': 60,
 'shopping cart': 61,
 'apparel': 62,
 'ottoman': 63

In [127]:
# We have to load the annotations of each frame of every video and check the classes present in each frame
# We have to maintain a dictionary with the info that is present in the previous frame
# Then, check the differences between the previous and the current frame and update the dictionary
# When 1 class appears, we have to add it to the dictionary a count of 1. If the class is already present, we have to add 1 to the count.
# When 1 class disappears, we have to add the obtained count to the final dictionary and restart the count of that class to 0.

# Final dict where each position of the list is one ocurrence of the class in the video
#   and the value is the number of frames that the class is present in the video in that ocurrence
ocurrence_vs_prevalence = {}
for k in classes_dict.values():
    ocurrence_vs_prevalence[k] = []

# Current frame info dict
current_frame_info = {}
for k in classes_dict.values():
    current_frame_info[k] = 0

# We have to load the annotations of each frame of every video and check the classes present in each frame
for idx_video, video_name in enumerate(video_names_list):

    # Get the folder path of the labels corresponding to the video
    labels_path = labels_folder_path / video_name
    # Get the list of the labels files
    labels_files_list = sorted(labels_path.glob('*.txt'))
    # Load the annotations of each frame of the video
    for idx_frame, label_file in enumerate(labels_files_list):
        with open(label_file) as f:
            annotations = f.readlines()
        for ann in annotations:
            idx_class = int(ann.split()[0])
            current_frame_info[idx_class] += 1
        
    break
        # # Get the frame number
        # frame_number = int(label_file.stem)
        # # Get the video path
        # video_path = videos_folder_path / f"{video_name}.mp4"
        # cap = cv2.VideoCapture(video_path.as_posix())
        # # Read the total number of frames
        # total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # # Read the frame
        # cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        # ret, frame = cap.read()
        # # Release the video capture object
        # cap.release()

        # # Get the classes present in the current frame
        # classes_present = [classes_dict[str(annotation['class_id'])] for annotation in annotations]
        # # Get the number of boxes per class
        # number_of_boxes_per_class = defaultdict(int)
        # for class_name in classes_present:
        #     number_of_boxes_per_class[class_name] += 1


In [128]:
current_frame_info

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 1,
 5: 1,
 6: 2,
 7: 0,
 8: 1,
 9: 0,
 10: 2,
 11: 0,
 12: 0,
 13: 0,
 14: 2451,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 25,
 21: 0,
 22: 1,
 23: 0,
 24: 0,
 25: 4,
 26: 2,
 27: 454,
 28: 0,
 29: 0,
 30: 1,
 31: 6,
 32: 0,
 33: 23,
 34: 0,
 35: 1,
 36: 136,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 19,
 43: 0,
 44: 0,
 45: 2,
 46: 0,
 47: 0,
 48: 0,
 49: 7,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 3,
 55: 0,
 56: 60,
 57: 0,
 58: 131,
 59: 122,
 60: 4,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 89,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 51,
 83: 0,
 84: 0,
 85: 101,
 86: 0,
 87: 0,
 88: 0,
 89: 17,
 90: 0,
 91: 0,
 92: 0,
 93: 11,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 2,
 109: 0,
 110: 0,
 111: 0,
 112: 0}

In [118]:
current_frame_info = {}
for k in classes_dict.values():
    current_frame_info[k] = 0

In [105]:
annotations

['14 0.3142361111111111 0.5023148148148148 0.03298611111111111 0.18055555555555555\n',
 '14 0.3350694444444444 0.5162037037037037 0.04513888888888889 0.18055555555555555\n',
 '14 0.4448784722222222 0.6574074074074074 0.20052083333333334 0.6851851851851852\n',
 '71 0.4454752604166667 0.6220100308641974 0.13346354166666666 0.36747685185185175\n',
 '27 0.7162905092592595 0.5546553497942387 0.31756365740740744 0.23019547325102874\n']

In [98]:
labels_files_list

[]

In [96]:
annotations

NameError: name 'annotations' is not defined

## Check which frames of the videos correspond to the annotated images (1 annotated frame every 60 aproximately)

In [ ]:
# Load image from .jpg file
images_list = []
for frame_image in frames_path_list:
    print(frame_image)
    images_list.append(np.array(Image.open(frame_image)))

In [76]:
# Step 2: Load the video file
video_path = videos_folder_path / f'{video_name}.mp4'
cap = cv2.VideoCapture(video_path.as_posix())

frame_number = 0
idx_frame = 0
selected_frames = [64, 96, 128, 160, 192] # Example frame numbers to visualize
selected_frames = list(np.arange(40, 1500, 1))

image = images_list[idx_frame]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop if there are no frames left to read
    
    # Step 4: Check if the current frame is one of the selected frames
    #if frame_number in selected_frames:
    # Step 5: Convert the frame from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)  # Convert to PIL Image

    # Convert to numpy array
    img = np.array(img)

    if (img == image).all():
        step_number = frames_path_list[idx_frame].stem[-5:]
        print(f'{frame_number}: Found frame corresponding to {step_number} ({(int(step_number) * 30)-1})')
        idx_frame += 1
        image = images_list[idx_frame]

        # Compare with another image
        
        # # Step 6: Visualize the frame
        # plt.figure(figsize=(10, 5))
        # plt.imshow(img)
        # plt.title(f'Frame {frame_number}')
        # plt.axis('off')  # Hide axis
        # plt.show()
    
    frame_number += 1

# Release the video capture object
cap.release()

59: Found frame corresponding to 00002 (59)
119: Found frame corresponding to 00004 (119)
179: Found frame corresponding to 00006 (179)
299: Found frame corresponding to 00010 (299)
359: Found frame corresponding to 00012 (359)
419: Found frame corresponding to 00014 (419)
479: Found frame corresponding to 00016 (479)
539: Found frame corresponding to 00018 (539)
599: Found frame corresponding to 00020 (599)
659: Found frame corresponding to 00022 (659)
719: Found frame corresponding to 00024 (719)
779: Found frame corresponding to 00026 (779)
839: Found frame corresponding to 00028 (839)
899: Found frame corresponding to 00030 (899)
959: Found frame corresponding to 00032 (959)
1019: Found frame corresponding to 00034 (1019)
1079: Found frame corresponding to 00036 (1079)
1139: Found frame corresponding to 00038 (1139)
1199: Found frame corresponding to 00040 (1199)


KeyboardInterrupt: 

In [19]:
video_images_path = images_path / video_name
video_labels_path = labels_path / video_name

print(len(list(video_images_path.iterdir())))
print(len(list(video_labels_path.iterdir())))

363
363


In [20]:
print(len(list((root_path / 'val' / 'Raw' / video_name).iterdir())))

363


In [23]:
print(len(list((root_path / 'images' / 'val').iterdir())))

7
